# Debug Runt Transform
template for testing/debugging runt transforms

In [1]:
import sys
import os
from os import sep
from os.path import dirname, realpath
from pathlib import Path
import logging
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)

def get_cwd(fname, subdir, crunch_dir=realpath(Path.home()) +sep +'crunch' +sep):
    """
    Convenience function to make a directory string for the current file based on inputs.
    Jupyter Notebook in Anaconda invokes the Python interpreter in Anaconda's subdirectory
    which is why changing sys.argv[0] is necessary. In the future a better way to do this
    should be preferred..
    """
    return crunch_dir +subdir +fname
    
def fix_path(cwd):
    """
    Convenience function to fix argv and python path so that jupyter notebook can run the same as
    any script in crunch.
    """
    sys.argv[0] = cwd
    module_path = os.path.abspath(os.path.join('..'))
    if module_path not in sys.path:
        sys.path.append(module_path)

fix_path(get_cwd('test.ipynb', 'recon' +sep))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from numba import jit, vectorize
from dask import delayed, dataframe as dd

from ipywidgets import interact, interactive, fixed
from IPython.display import display

pd.set_option("display.max_rows", 100)
pd.set_option('display.max_columns', 50)

from common_util import MUTATE_DIR, DT_HOURLY_FREQ, DT_CAL_DAILY_FREQ
from common_util import zdiv, load_json, dump_json, remove_dups_list, list_get_dict, is_empty_df, search_df, benchmark
from data.data_api import DataAPI
from data.access_util import df_getters as dg, col_subsetters2 as cs2
from mutate.common import default_runt_dir_name, default_trfs_dir_name
from mutate.runt_util import *
from mutate.runt import fill_defaults, get_variants, get_row_mask_keychain, get_desc_pfx, make_runt_entry

DEBUG:matplotlib:$HOME=/home/kevin
DEBUG:matplotlib:matplotlib data path /home/kevin/miniconda3/lib/python3.6/site-packages/matplotlib/mpl-data
DEBUG:matplotlib:loaded rc file /home/kevin/miniconda3/lib/python3.6/site-packages/matplotlib/mpl-data/matplotlibrc
DEBUG:matplotlib:matplotlib version 2.2.2
DEBUG:matplotlib:interactive is False
DEBUG:matplotlib:platform is linux
DEBUG:matplotlib:loaded modules: ['builtins', 'sys', '_frozen_importlib', '_imp', '_warnings', '_thread', '_weakref', '_frozen_importlib_external', '_io', 'marshal', 'posix', 'zipimport', 'encodings', 'codecs', '_codecs', 'encodings.aliases', 'encodings.utf_8', '_signal', '__main__', 'encodings.latin_1', 'io', 'abc', '_weakrefset', '_bootlocale', '_locale', 'site', 'os', 'errno', 'stat', '_stat', 'posixpath', 'genericpath', 'os.path', '_collections_abc', '_sitebuiltins', 'sysconfig', '_sysconfigdata_m_linux_x86_64-linux-gnu', 'types', 'functools', '_functools', 'collections', 'operator', '_operator', 'keyword', 'heapq

DEBUG:matplotlib:CACHEDIR=/home/kevin/.cache/matplotlib
DEBUG:matplotlib.font_manager:Using fontManager instance from /home/kevin/.cache/matplotlib/fontList.json
DEBUG:matplotlib.backends:backend module://ipykernel.pylab.backend_inline version unknown
DEBUG:matplotlib.backends:backend module://ipykernel.pylab.backend_inline version unknown


## General

### Set Logging Level and Date Range

In [2]:
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
date_range = {
    'id': ('lt', 2018)
}

### Set Runt and Transform Directories

In [3]:
runt_dir_name = default_runt_dir_name
trfs_dir_name = default_trfs_dir_name

runt_dir = MUTATE_DIR +runt_dir_name
trfs_dir = runt_dir +trfs_dir_name

### Load all transforms

In [4]:
trf_defaults = load_json('trf_defaults.json', dir_path=runt_dir)
graph = load_json('graph.json', dir_path=runt_dir)
trfs = {}

logging.info('loading step settings...')
for fname in os.listdir(trfs_dir):
    trf = load_json(fname, dir_path=trfs_dir)
    trfs[trf['meta']['name']] = trf

INFO:root:loading step settings...


### Define Test Step Function

In [5]:
def test_step(step_info, date_range, overwrite_fn={}):
    meta, fn, var, rm, src, dst = step_info['meta'], step_info['fn'], step_info['var'], step_info['rm'], step_info['src'], step_info['dst']

    # Loading transform, apply, and frequency settings
    ser_fn = overwrite_fn['ser_fn'] if ('ser_fn' in overwrite_fn) else RUNT_FN_TRANSLATOR[fn['ser_fn']]
    rtype_fn = overwrite_fn['df_fn'] if ('df_fn' in overwrite_fn) else RUNT_TYPE_TRANSLATOR[fn['df_fn']]
    freq = overwrite_fn['freq'] if ('freq' in overwrite_fn) else RUNT_FREQ_TRANSLATOR[fn['freq']]
    res_freq = RUNT_FREQ_TRANSLATOR[meta['res_freq']]

    # Making all possible parameter combinations
    variants = get_variants(var, meta['var_fmt'])
    logging.debug('variants: ' +str(variants))

    # Loading row mask, if any
    if (rm is not None):
        rm_dg, rm_cs = list_get_dict(dg, rm), list_get_dict(cs2, rm)
        rm_paths, rm_recs, rm_dfs = DataAPI.load_from_dg(rm_dg, rm_cs)
        rm_keys = [remove_dups_list([key_chain[i] for key_chain in rm_paths]) for i in range(len(rm_paths[0]))]

    # Loading input data
    src_dg, src_cs = list_get_dict(dg, src), list_get_dict(cs2, src)
    src_paths, src_recs, src_dfs = DataAPI.load_from_dg(src_dg, src_cs)
    logging.debug('src_paths[0] ' +str(src_paths[0]))
    logging.debug('src_paths[-1] ' +str(src_paths[-1]))

    # Run transforms on inputs
    for key_chain in src_paths:
        logging.info('data: ' +str('_'.join(key_chain)))
        src_rec, src_df = list_get_dict(src_recs, key_chain), list_get_dict(src_dfs, key_chain)
        src_df = src_df.loc[search_df(src_df, date_range), :].dropna(axis=0, how='all')

        # Masking rows in src from row mask
        if (rm is not None):
            rm_key_chain = get_row_mask_keychain(key_chain, rm_keys)
            rm_df = list_get_dict(rm_dfs, rm_key_chain).dropna()
            not_in_src = rm_df.index.difference(src_df.index)
            logging.debug('row mask: ' +str('_'.join(rm_key_chain)))
            if (len(not_in_src)>0):
                logging.debug('rm_idx - src_idx: ' +str(not_in_src))
                src_df = src_df.loc[src_df.index & rm_df.index, :].dropna(axis=0, how='all')
            else:
                src_df = src_df.loc[rm_df.index, :].dropna(axis=0, how='all')

        logging.debug('pre_transform: ' +str(src_df))

        # Running variants of the transform
        for variant in variants:
            transform_fn = ser_fn(**variant)
            runted_df = rtype_fn(src_df, transform_fn, freq)
            print(variant)
            desc_sfx = meta['rec_fmt'].format(**variant)
            desc_pfx = get_desc_pfx(key_chain, src_rec)
            desc = '_'.join([desc_pfx, desc_sfx])

            if (meta['mtype_from']=='name'):       mutate_type = meta['name']
            elif (meta['mtype_from']=='rec_fmt'):  mutate_type = desc_sfx

            assert(not is_empty_df(runted_df))
            entry = make_runt_entry(desc, res_freq, mutate_type, src_rec)
            logging.info('dumping ' +desc +'...')
#             logging.debug('post_transform: ' +str(runted_df))
            yield entry, runted_df

## Transform Debug

### Set Transform
Fill it in (default is day moving average)

In [6]:
trf_name = 'dffracdiff'

### Transform Imports

In [7]:
from mutate.fracdiff import *

### Test Section

In [8]:
logging.basicConfig(stream=sys.stdout, level=logging.INFO)

In [9]:
trf = trfs[trf_name]
step = fill_defaults(trf, trf_defaults)

### Step Overwrites

### Overwrites

In [10]:
def get_weights_FFD_custom(d, size=None, thresh=None):
    weights, weight_idx, next_weight = [], 0, 1 # First weight is one (the latest value which will be differenced)

    while (abs(next_weight) >= thresh):
        weights.append(next_weight); weight_idx += 1;
        
        # Calculate next_weight: wₖ = -wₖ₋₁ * ((d-k+1) / k)
        weight_multiplier = zdiv((d-weight_idx+1), weight_idx)
        next_weight = -weights[-1] * weight_multiplier

    weights = np.array(weights[::-1]).reshape(-1, 1)
    logging.info('weights: ' +str(weights.T))

    return weights

In [11]:
def fracdiff(d, size, thresh, method):
    weights = get_weights_FFD_custom(d, size=size, thresh=thresh)
    if (method == 'EFD'):		# Expanding Window
        # TODO - Implement
        # Determine initial calcs to be skipped based on weight-loss threshold
        weight_changes = np.cumsum(abs(weights))
        weight_changes /= weight_changes[-1]
        skip = weight_changes[weight_changes > thresh].shape[0]

    elif (method == 'FFD'):		# Fixed Window
        pass

    dot_weights_fn = lambda ser: ser.dot(weights)
    return lambda ser: ser.dropna().rolling(window=len(weights), min_periods=len(weights)).apply(dot_weights_fn)

In [12]:
# ovs = {}
# ovs['ser_fn'] = fracdiff

In [13]:
for runt_rec, runt_df in test_step(step, date_range, overwrite_fn={}):
    print('XXXXXXXX')

DEBUG:root:variants: [{'d': 0.2, 'size': None, 'thresh': 0.01}, {'d': 0.2, 'size': None, 'thresh': 0.1}, {'d': 0.4, 'size': None, 'thresh': 0.01}, {'d': 0.4, 'size': None, 'thresh': 0.1}, {'d': 0.5, 'size': None, 'thresh': 0.01}, {'d': 0.5, 'size': None, 'thresh': 0.1}, {'d': 0.6, 'size': None, 'thresh': 0.01}, {'d': 0.6, 'size': None, 'thresh': 0.1}, {'d': 0.8, 'size': None, 'thresh': 0.01}, {'d': 0.8, 'size': None, 'thresh': 0.1}, {'d': 1.0, 'size': None, 'thresh': 0.01}, {'d': 1.0, 'size': None, 'thresh': 0.1}, {'d': 1.5, 'size': None, 'thresh': 0.01}, {'d': 1.5, 'size': None, 'thresh': 0.1}, {'d': 2.0, 'size': None, 'thresh': 0.01}, {'d': 2.0, 'size': None, 'thresh': 0.1}]
DEBUG:root:src_paths[0] ['dow_jones', 'drf', 'raw_pba']
DEBUG:root:src_paths[-1] ['russell_2000', 'drl', 'thresh_vol']
INFO:root:data: dow_jones_drf_raw_pba
DEBUG:root:pre_transform:                            pba_open  pba_close  pba_avgPrice  pba_numTrades
id                                                     

DEBUG:root:weights: [[-0.01097303 -0.01246935 -0.01438771 -0.01692672 -0.0204288  -0.025536
  -0.0336     -0.048      -0.08       -0.2         1.        ]]
{'d': 0.2, 'size': None, 'thresh': 0.01}
INFO:root:dumping raw_pba_drf_dffracdiff(0.2000,0.0100)...
XXXXXXXX
DEBUG:root:weights: [[-0.2  1. ]]
{'d': 0.2, 'size': None, 'thresh': 0.1}
INFO:root:dumping raw_pba_drf_dffracdiff(0.2000,0.1000)...
XXXXXXXX
DEBUG:root:weights: [[-0.01100641 -0.01279816 -0.01515571 -0.01837056 -0.0229632  -0.029952
  -0.0416     -0.064      -0.12       -0.4         1.        ]]
{'d': 0.4, 'size': None, 'thresh': 0.01}
INFO:root:dumping raw_pba_drf_dffracdiff(0.4000,0.0100)...
XXXXXXXX
DEBUG:root:weights: [[-0.12 -0.4   1.  ]]
{'d': 0.4, 'size': None, 'thresh': 0.1}
INFO:root:dumping raw_pba_drf_dffracdiff(0.4000,0.1000)...
XXXXXXXX
DEBUG:root:weights: [[-0.01091003 -0.01309204 -0.01611328 -0.02050781 -0.02734375 -0.0390625
  -0.0625     -0.125      -0.5         1.        ]]


/home/kevin/crunch/mutate/runt_util.py:56: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  return lambda ser: ser.dropna().rolling(window=len(weights), min_periods=len(weights)).apply(dot_weights_fn)


{'d': 0.5, 'size': None, 'thresh': 0.01}
INFO:root:dumping raw_pba_drf_dffracdiff(0.5000,0.0100)...
XXXXXXXX
DEBUG:root:weights: [[-0.125 -0.5    1.   ]]
{'d': 0.5, 'size': None, 'thresh': 0.1}
INFO:root:dumping raw_pba_drf_dffracdiff(0.5000,0.1000)...
XXXXXXXX
DEBUG:root:weights: [[-0.01034035 -0.01292544 -0.0167552  -0.022848   -0.0336     -0.056
  -0.12       -0.6         1.        ]]
{'d': 0.6, 'size': None, 'thresh': 0.01}
INFO:root:dumping raw_pba_drf_dffracdiff(0.6000,0.0100)...
XXXXXXXX
DEBUG:root:weights: [[-0.12 -0.6   1.  ]]
{'d': 0.6, 'size': None, 'thresh': 0.1}
INFO:root:dumping raw_pba_drf_dffracdiff(0.6000,0.1000)...
XXXXXXXX
DEBUG:root:weights: [[-0.011264 -0.0176   -0.032    -0.08     -0.8       1.      ]]
{'d': 0.8, 'size': None, 'thresh': 0.01}
INFO:root:dumping raw_pba_drf_dffracdiff(0.8000,0.0100)...
XXXXXXXX
DEBUG:root:weights: [[-0.8  1. ]]
{'d': 0.8, 'size': None, 'thresh': 0.1}
INFO:root:dumping raw_pba_drf_dffracdiff(0.8000,0.1000)...
XXXXXXXX
DEBUG:root:weig

DEBUG:root:weights: [[-0.01097303 -0.01246935 -0.01438771 -0.01692672 -0.0204288  -0.025536
  -0.0336     -0.048      -0.08       -0.2         1.        ]]
{'d': 0.2, 'size': None, 'thresh': 0.01}
INFO:root:dumping raw_pba_drf_dffracdiff(0.2000,0.0100)...
XXXXXXXX
DEBUG:root:weights: [[-0.2  1. ]]
{'d': 0.2, 'size': None, 'thresh': 0.1}
INFO:root:dumping raw_pba_drf_dffracdiff(0.2000,0.1000)...
XXXXXXXX
DEBUG:root:weights: [[-0.01100641 -0.01279816 -0.01515571 -0.01837056 -0.0229632  -0.029952
  -0.0416     -0.064      -0.12       -0.4         1.        ]]
{'d': 0.4, 'size': None, 'thresh': 0.01}
INFO:root:dumping raw_pba_drf_dffracdiff(0.4000,0.0100)...
XXXXXXXX
DEBUG:root:weights: [[-0.12 -0.4   1.  ]]
{'d': 0.4, 'size': None, 'thresh': 0.1}
INFO:root:dumping raw_pba_drf_dffracdiff(0.4000,0.1000)...
XXXXXXXX
DEBUG:root:weights: [[-0.01091003 -0.01309204 -0.01611328 -0.02050781 -0.02734375 -0.0390625
  -0.0625     -0.125      -0.5         1.        ]]
{'d': 0.5, 'size': None, 'thresh'

...

DEBUG:root:weights: [[-0.01097303 -0.01246935 -0.01438771 -0.01692672 -0.0204288  -0.025536
  -0.0336     -0.048      -0.08       -0.2         1.        ]]
{'d': 0.2, 'size': None, 'thresh': 0.01}
INFO:root:dumping thresh_pba_drl_dffracdiff(0.2000,0.0100)...
XXXXXXXX
DEBUG:root:weights: [[-0.2  1. ]]
{'d': 0.2, 'size': None, 'thresh': 0.1}
INFO:root:dumping thresh_pba_drl_dffracdiff(0.2000,0.1000)...
XXXXXXXX
DEBUG:root:weights: [[-0.01100641 -0.01279816 -0.01515571 -0.01837056 -0.0229632  -0.029952
  -0.0416     -0.064      -0.12       -0.4         1.        ]]
{'d': 0.4, 'size': None, 'thresh': 0.01}
INFO:root:dumping thresh_pba_drl_dffracdiff(0.4000,0.0100)...
XXXXXXXX
DEBUG:root:weights: [[-0.12 -0.4   1.  ]]
{'d': 0.4, 'size': None, 'thresh': 0.1}
INFO:root:dumping thresh_pba_drl_dffracdiff(0.4000,0.1000)...
XXXXXXXX
DEBUG:root:weights: [[-0.01091003 -0.01309204 -0.01611328 -0.02050781 -0.02734375 -0.0390625
  -0.0625     -0.125      -0.5         1.        ]]
{'d': 0.5, 'size': No

DEBUG:root:weights: [[-0.01097303 -0.01246935 -0.01438771 -0.01692672 -0.0204288  -0.025536
  -0.0336     -0.048      -0.08       -0.2         1.        ]]
{'d': 0.2, 'size': None, 'thresh': 0.01}
INFO:root:dumping thresh_vol_drl_dffracdiff(0.2000,0.0100)...
XXXXXXXX
DEBUG:root:weights: [[-0.2  1. ]]
{'d': 0.2, 'size': None, 'thresh': 0.1}
INFO:root:dumping thresh_vol_drl_dffracdiff(0.2000,0.1000)...
XXXXXXXX
DEBUG:root:weights: [[-0.01100641 -0.01279816 -0.01515571 -0.01837056 -0.0229632  -0.029952
  -0.0416     -0.064      -0.12       -0.4         1.        ]]
{'d': 0.4, 'size': None, 'thresh': 0.01}
INFO:root:dumping thresh_vol_drl_dffracdiff(0.4000,0.0100)...
XXXXXXXX
DEBUG:root:weights: [[-0.12 -0.4   1.  ]]
{'d': 0.4, 'size': None, 'thresh': 0.1}
INFO:root:dumping thresh_vol_drl_dffracdiff(0.4000,0.1000)...
XXXXXXXX
DEBUG:root:weights: [[-0.01091003 -0.01309204 -0.01611328 -0.02050781 -0.02734375 -0.0390625
  -0.0625     -0.125      -0.5         1.        ]]
{'d': 0.5, 'size': No

DEBUG:root:weights: [[-0.01097303 -0.01246935 -0.01438771 -0.01692672 -0.0204288  -0.025536
  -0.0336     -0.048      -0.08       -0.2         1.        ]]
{'d': 0.2, 'size': None, 'thresh': 0.01}
INFO:root:dumping thresh_vol_drl_dffracdiff(0.2000,0.0100)...
XXXXXXXX
DEBUG:root:weights: [[-0.2  1. ]]
{'d': 0.2, 'size': None, 'thresh': 0.1}
INFO:root:dumping thresh_vol_drl_dffracdiff(0.2000,0.1000)...
XXXXXXXX
DEBUG:root:weights: [[-0.01100641 -0.01279816 -0.01515571 -0.01837056 -0.0229632  -0.029952
  -0.0416     -0.064      -0.12       -0.4         1.        ]]
{'d': 0.4, 'size': None, 'thresh': 0.01}
INFO:root:dumping thresh_vol_drl_dffracdiff(0.4000,0.0100)...
XXXXXXXX
DEBUG:root:weights: [[-0.12 -0.4   1.  ]]
{'d': 0.4, 'size': None, 'thresh': 0.1}
INFO:root:dumping thresh_vol_drl_dffracdiff(0.4000,0.1000)...
XXXXXXXX
DEBUG:root:weights: [[-0.01091003 -0.01309204 -0.01611328 -0.02050781 -0.02734375 -0.0390625
  -0.0625     -0.125      -0.5         1.        ]]
{'d': 0.5, 'size': No

DEBUG:root:weights: [[-0.01097303 -0.01246935 -0.01438771 -0.01692672 -0.0204288  -0.025536
  -0.0336     -0.048      -0.08       -0.2         1.        ]]
{'d': 0.2, 'size': None, 'thresh': 0.01}
INFO:root:dumping thresh_vol_drl_dffracdiff(0.2000,0.0100)...
XXXXXXXX
DEBUG:root:weights: [[-0.2  1. ]]
{'d': 0.2, 'size': None, 'thresh': 0.1}
INFO:root:dumping thresh_vol_drl_dffracdiff(0.2000,0.1000)...
XXXXXXXX
DEBUG:root:weights: [[-0.01100641 -0.01279816 -0.01515571 -0.01837056 -0.0229632  -0.029952
  -0.0416     -0.064      -0.12       -0.4         1.        ]]
{'d': 0.4, 'size': None, 'thresh': 0.01}
INFO:root:dumping thresh_vol_drl_dffracdiff(0.4000,0.0100)...
XXXXXXXX
DEBUG:root:weights: [[-0.12 -0.4   1.  ]]
{'d': 0.4, 'size': None, 'thresh': 0.1}
INFO:root:dumping thresh_vol_drl_dffracdiff(0.4000,0.1000)...
XXXXXXXX
DEBUG:root:weights: [[-0.01091003 -0.01309204 -0.01611328 -0.02050781 -0.02734375 -0.0390625
  -0.0625     -0.125      -0.5         1.        ]]
{'d': 0.5, 'size': No

DEBUG:root:weights: [[-0.01097303 -0.01246935 -0.01438771 -0.01692672 -0.0204288  -0.025536
  -0.0336     -0.048      -0.08       -0.2         1.        ]]
{'d': 0.2, 'size': None, 'thresh': 0.01}
INFO:root:dumping thresh_vol_drl_dffracdiff(0.2000,0.0100)...
XXXXXXXX
DEBUG:root:weights: [[-0.2  1. ]]
{'d': 0.2, 'size': None, 'thresh': 0.1}
INFO:root:dumping thresh_vol_drl_dffracdiff(0.2000,0.1000)...
XXXXXXXX
DEBUG:root:weights: [[-0.01100641 -0.01279816 -0.01515571 -0.01837056 -0.0229632  -0.029952
  -0.0416     -0.064      -0.12       -0.4         1.        ]]
{'d': 0.4, 'size': None, 'thresh': 0.01}
INFO:root:dumping thresh_vol_drl_dffracdiff(0.4000,0.0100)...
XXXXXXXX
DEBUG:root:weights: [[-0.12 -0.4   1.  ]]
{'d': 0.4, 'size': None, 'thresh': 0.1}
INFO:root:dumping thresh_vol_drl_dffracdiff(0.4000,0.1000)...
XXXXXXXX
DEBUG:root:weights: [[-0.01091003 -0.01309204 -0.01611328 -0.02050781 -0.02734375 -0.0390625
  -0.0625     -0.125      -0.5         1.        ]]
{'d': 0.5, 'size': No